# Designing Novel Toehold Sensors

# Steps 
1. Define a specific target sequence
2. Divide the sequence in windows of the same length of the trigger (36nt)
3. For each possible trigger, define a toehold sensor sequence
4. Filter sequences that contain stop codons
5. Evaluate sequence properties using NUPACK and rank them

### Import dependencies

In [1]:
import os
import os.path
from subprocess import Popen, PIPE
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import statsmodels.api as sm
from skbio import Sequence

/home/anibal/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


### Define library functions

In [2]:
def reversed_complement(sequence):
    mapping = {'A': 'U', 'G': 'C', 'U': 'A', 'C': 'G'}
    sequence_upper = sequence.upper()

    complement = ''
    for c in sequence_upper:
        complement += mapping[c]

    # reverse the sequence
    return complement[::-1]

def split_sequence(sequence, window):
    sequences = []
    limit = len(sequence) - window + 1

    for i in range(0, limit):
        sequences.append(sequence[i:window + i])

    return sequences

def no_stop(sequence):
    stop = ['UAA', 'UAG', 'UGA']

    for i in range(0, len(sequence), 3):
        if sequence[i:i + 3] in stop:
            return False

    return True

def possible_toehold_B(reg, rev):
    
    #to avoid any posterior conflicts please write loops and linkers in RNA form!
    
    loop = 'GGACUUUAGAACAGAGGAGAUAAAGAUG' # The last three nucleotides is AUG, the start codon.
    
    linker = 'ACCUGGCGGCAGCGCAAGAAG'#( from Green paper 2019 se, NOTE THAT IT DOES NOT HAVE THE MOCLO SITE C,  AATG)
    
    #other linkers = "AACCUGGCGGCAGCGCAAGAAGAUGCGUAAA" 
    toeholds = []

    for n in ['A', 'G', 'U', 'C']: 
        if no_stop(reg[0:11] + linker):
            toeholds.append(rev + loop + reg[0:11] + linker)
            return toeholds
      
    
    toeholds.append("STOP")
    return toeholds


### Step 1: Define a target sequence
In this case, we use the  5' of the glycoprotein 1 from PVY:

In [3]:
#seq = "GGAGTTTGGGTTATGATGGATGGAGATGAACAAGTCGAATACCCACTGAAACCAATCGTTGAGAATGCAAAACCAACACTTAGGCAAATCATGGCACATTTCTCAGATGTTGCAGAAGCGTATATAGAAATGCGCAACAAAAAGGAACCATATATGCCACGATATGGTTTAGTTCGTAATCTGCGCGATGGAAGTTTGGCTCGCTATGCTTTTGACTTTTATG"
#seq= "TGCAATGGGATAGAGCTGATCTGCCAGAGCACAGATTAGAAGCGATTTGTGCAGCAATGATAGAATCCTGGGGTTATTTTGAGTTAACGCACCAAATCAGGAGATTCTACTCATGGTTGTTGCAACAGCAACCTTTTTCAACGATAGC"
#seq= "GGAGTTTGGGTTATGATGGATGGAGATGAACAAGTCGAATACCCACTGAAACCAATCGTTGAGAATGCAAAACCAACACTTAGGCAAATCATGGCACATTTCTCAGATGTTGCAGAAGCGTATATAGAAATGCGCAACAAAAAGGAACCATATATGCCACGATATGGTTTAGTTCGTAATCTGCGCGATGGAAGTTTGGCTCGCTATGCTTTTGACTTTTATGAAGTTAC"
seq = "gtactgccaactggatccttcgcgggacgtcctttgtttacgtcccgtcggcgctgaatcccgcggacgacccctctcggggccgcttgggactctctcgtccccttctccgtctgccgttccagccgaccacggggcgcacctctctttacgcggtctccccgtctgtgccttctcatctgccggtccgtgtgcacttcgcttcacctctgcacgttgcatggagaccaccgtgaacgcccatcagatcctgcccaaggtcttacataagaggactcttggactcccagcaatgtcaacgaccgaccttgaggcctacttcaaagactgtgtgtttaaggactg"


Convert to RNA and determine the reverse complement.

In [4]:
processed_sequence = seq.upper().replace('T', 'U') #/.replace(' ', '')
rc = reversed_complement(processed_sequence)

### Step 2: Determine 36-nucleotide sub-sequences
To do this, we make all possible triggers for the direct and reversed complementary sequence. 
¿How many sub-sequences of 36 nt do we need to analyze ?

In [5]:
len(split_sequence(rc,36))

310

In [6]:
d_1 = {'Triggers': split_sequence(processed_sequence,36)}
df_1 = pd.DataFrame(data=d_1)
df_1["Sense"]="Direct"

d_2 = {'Triggers': split_sequence(rc,36)}
df_2 = pd.DataFrame(data=d_2)
df_2["Sense"]="Reversed Complement"
frames = [df_1, df_2]
result = pd.concat(frames)
result

,Triggers,Sense
0,GUACUGCCAACUGGAUCCUUCGCGGGACGUCCUUUG,Direct
1,UACUGCCAACUGGAUCCUUCGCGGGACGUCCUUUGU,Direct
2,ACUGCCAACUGGAUCCUUCGCGGGACGUCCUUUGUU,Direct
3,CUGCCAACUGGAUCCUUCGCGGGACGUCCUUUGUUU,Direct
4,UGCCAACUGGAUCCUUCGCGGGACGUCCUUUGUUUA,Direct
5,GCCAACUGGAUCCUUCGCGGGACGUCCUUUGUUUAC,Direct
6,CCAACUGGAUCCUUCGCGGGACGUCCUUUGUUUACG,Direct
7,CAACUGGAUCCUUCGCGGGACGUCCUUUGUUUACGU,Direct
8,AACUGGAUCCUUCGCGGGACGUCCUUUGUUUACGUC,Direct
9,ACUGGAUCCUUCGCGGGACGUCCUUUGUUUACGUCC,Direct


### Step 3: For each trigger, design a toehold sensor

In [7]:
toeholds=[]
for i in range(len(result.iloc[:,0])):
    toeholds.append((possible_toehold_B(result.iloc[i,0],reversed_complement(result.iloc[i,0])))[0])
a=pd.Series(toeholds)
result["Toehold Switch"]=a.values
result

,Triggers,Sense,Toehold Switch
0,GUACUGCCAACUGGAUCCUUCGCGGGACGUCCUUUG,Direct,CAAAGGACGUCCCGCGAAGGAUCCAGUUGGCAGUACGGACUUUAGA...
1,UACUGCCAACUGGAUCCUUCGCGGGACGUCCUUUGU,Direct,ACAAAGGACGUCCCGCGAAGGAUCCAGUUGGCAGUAGGACUUUAGA...
2,ACUGCCAACUGGAUCCUUCGCGGGACGUCCUUUGUU,Direct,STOP
3,CUGCCAACUGGAUCCUUCGCGGGACGUCCUUUGUUU,Direct,AAACAAAGGACGUCCCGCGAAGGAUCCAGUUGGCAGGGACUUUAGA...
4,UGCCAACUGGAUCCUUCGCGGGACGUCCUUUGUUUA,Direct,UAAACAAAGGACGUCCCGCGAAGGAUCCAGUUGGCAGGACUUUAGA...
5,GCCAACUGGAUCCUUCGCGGGACGUCCUUUGUUUAC,Direct,GUAAACAAAGGACGUCCCGCGAAGGAUCCAGUUGGCGGACUUUAGA...
6,CCAACUGGAUCCUUCGCGGGACGUCCUUUGUUUACG,Direct,CGUAAACAAAGGACGUCCCGCGAAGGAUCCAGUUGGGGACUUUAGA...
7,CAACUGGAUCCUUCGCGGGACGUCCUUUGUUUACGU,Direct,ACGUAAACAAAGGACGUCCCGCGAAGGAUCCAGUUGGGACUUUAGA...
8,AACUGGAUCCUUCGCGGGACGUCCUUUGUUUACGUC,Direct,GACGUAAACAAAGGACGUCCCGCGAAGGAUCCAGUUGGACUUUAGA...
9,ACUGGAUCCUUCGCGGGACGUCCUUUGUUUACGUCC,Direct,GGACGUAAACAAAGGACGUCCCGCGAAGGAUCCAGUGGACUUUAGA...


### Step 4: Remove sensors with STOP codons

In [8]:
df = result[result.iloc[:,2] != "STOP"]
df

,Triggers,Sense,Toehold Switch
0,GUACUGCCAACUGGAUCCUUCGCGGGACGUCCUUUG,Direct,CAAAGGACGUCCCGCGAAGGAUCCAGUUGGCAGUACGGACUUUAGA...
1,UACUGCCAACUGGAUCCUUCGCGGGACGUCCUUUGU,Direct,ACAAAGGACGUCCCGCGAAGGAUCCAGUUGGCAGUAGGACUUUAGA...
3,CUGCCAACUGGAUCCUUCGCGGGACGUCCUUUGUUU,Direct,AAACAAAGGACGUCCCGCGAAGGAUCCAGUUGGCAGGGACUUUAGA...
4,UGCCAACUGGAUCCUUCGCGGGACGUCCUUUGUUUA,Direct,UAAACAAAGGACGUCCCGCGAAGGAUCCAGUUGGCAGGACUUUAGA...
5,GCCAACUGGAUCCUUCGCGGGACGUCCUUUGUUUAC,Direct,GUAAACAAAGGACGUCCCGCGAAGGAUCCAGUUGGCGGACUUUAGA...
6,CCAACUGGAUCCUUCGCGGGACGUCCUUUGUUUACG,Direct,CGUAAACAAAGGACGUCCCGCGAAGGAUCCAGUUGGGGACUUUAGA...
7,CAACUGGAUCCUUCGCGGGACGUCCUUUGUUUACGU,Direct,ACGUAAACAAAGGACGUCCCGCGAAGGAUCCAGUUGGGACUUUAGA...
8,AACUGGAUCCUUCGCGGGACGUCCUUUGUUUACGUC,Direct,GACGUAAACAAAGGACGUCCCGCGAAGGAUCCAGUUGGACUUUAGA...
9,ACUGGAUCCUUCGCGGGACGUCCUUUGUUUACGUCC,Direct,GGACGUAAACAAAGGACGUCCCGCGAAGGAUCCAGUGGACUUUAGA...
10,CUGGAUCCUUCGCGGGACGUCCUUUGUUUACGUCCC,Direct,GGGACGUAAACAAAGGACGUCCCGCGAAGGAUCCAGGGACUUUAGA...


### Step 5: Evaluate sequence properties using NUPACK 

Some scoring functions from Ma, D, et al. 2018):
- Three-parameter fit (R2 = 0.57):
- Fold change = –71.7 dfull_sensor  – 49.1 dactive_sensor – 22.6 dbinding_site + 54.3

- Four-parameter fit (R2 = 0.60):
- Fold change = –93.2 dfull_sensor – 43.3 dactive_sensor – 22.1 dbinding_site – 9.4 dmin_target + 61.3

Definition of the parameters :
- Ensamble Defect: represents the average number of incorrectly paired nucleotides at equilibrium, evaluated over the ensemble of the complex
- dfull_sensor:  Ensamble defect for the full toehold switch sequence and structure 
- dactive_sensor: Ensemble defect was calculated directly from the sequence from the first base of the loop sequence. A completely single-stranded secondary structure was used for assessing design quality for dactive_sensor.
- dbinding_site: Ensemble defect was calculated in an analogous manner using the pairwise binding probabilities of the complete target RNA sequence and specifying a completely single-stranded ideal secondary structure in the binding site region.

#### Library functions
- Calculation of the minimum free energy (MFE) secondary structure of a singular RNA sequence
- NUPACK analysis

In [21]:
def DG(sequence,result_path,wait):
    file = open('{}pipo.in'.format(result_path), 'w')
    file.write("{}\n".format(sequence))
    file.close()
    final=[]
    semi_final=[]

    Popen(["mfe -T 29 {}pipo".format(result_path)],shell=True, stdout=PIPE)
    time.sleep(wait)
    lenght=len(sequence)
    with open("{}pipo.mfe".format(result_path)) as res:
        for r in res:
            r = r.strip('\n')
            if not r.startswith('%'):
                r = r.split('\t')
                semi_final.append(r)

    #final.append()                

    

    os.remove("{}pipo.mfe".format(result_path,))
    os.remove("{}pipo.in".format(result_path))
    return (float(semi_final[2][0]))
    
def complex_defect(sequence, secondary, result_path):
    file = open('{}toeh.in'.format(result_path), 'w')
    file.write("{}\n".format(sequence))
    file.write("{}".format(secondary))
    file.close()

    defect_toeh = 0
    count = 0
    with Popen(["complexdefect", "{}toeh".format(result_path)], stdout=PIPE) as proc:
        res = (proc.stdout.read()).decode("utf-8").split('\n')
        for l in res:
            count += 1
            if count == 16:
                defect_toeh = float(l)

    os.remove("{}toeh.in".format(result_path))
    return defect_toeh

def nupack_analysis(sequence, secondary_sensor,  window, sensor_type, result_path):
    list_for_table = []

    processed_sequence = sequence.upper().replace('T', 'U').replace(' ', '')
    reg_sequences = split_sequence(processed_sequence, window)
    rev_comp_sequences = [reversed_complement(s) for s in reg_sequences]

    if sensor_type == 'A':
        target_toehold_map = possible_toehold_A(reg_sequences, rev_comp_sequences)
    else:
        target_toehold_map = possible_toehold_B(reg_sequences, rev_comp_sequences)

    sequence = sequence.upper().replace('T', 'U')
    single_streadness_sequence = single_streadness(sequence, result_path, wait=6)
    for target, toehold in target_toehold_map.items():
        id = sequence.index(target)

        target_defect = sum(single_streadness_sequence[id:id+36])/36
        toehold_defect = sum(single_streadness(toehold, result_path)[0:30])/30
        sensor_defect = complex_defect(toehold, secondary_sensor, result_path)

        score = 5*(1-target_defect) + 4*(1-toehold_defect) + 3*sensor_defect

        list_for_table.append(tuple([target[0:36], toehold, 1-target_defect, 1-toehold_defect, sensor_defect, score]))

    return list_for_table

linear_Structure="............................................................"
linear_Structure_25="........................."
secondary_sensor_B = '.........................(((((((((((...(((((............)))))...))))))))))).....................'
secondary_2=".......................................(((((............)))))....................................+...................................."

#### Calculation of the thermodynamic parameters for each design

In [22]:


##fromNUPACK

dfull_sensor=[]

for i in range (len(df.Triggers)):
    dfull_sensor.append((complex_defect(df.iloc[i,2],secondary_sensor_B,"Temp/")))
dfull_sensor
P1=pd.Series(dfull_sensor)
df["dfull_sensor"]=P1.values

dactive_sensor=[]
for i in range (len(df.Triggers)):
   
    dactive_sensor.append((complex_defect(df.iloc[i,2][36::],linear_Structure,"Temp/")))
P2=pd.Series(dactive_sensor)
df["dactive_sensor"]=P2.values

dbinding_site=[]
for i in range (len(df.Triggers)):
   
    dbinding_site.append((complex_defect(df.iloc[i,2][0:25],linear_Structure_25,"Temp/")))
P3=pd.Series(dbinding_site)
df["dbinding_site"]=P3.values

Dg_RBS_linker=[]
for i in range(len(df.iloc[:,3])):
    RBS_linker=df.iloc[i,2][48:96]
    Dg_RBS_linker.append(DG(RBS_linker,"Temp/",2))
P5=pd.Series(Dg_RBS_linker)
df["Dg_RBS_linker"]=P5.values



/home/anibal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/home/anibal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/anibal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [14]:
score=[]
for i in range (len(df.Triggers)):
    score.append(54.3 -71.7*df.iloc[i,3] -49.1*df.iloc[i,4] -22.6**df.iloc[i,5])
P4=pd.Series(score)
df["score"]=P4.values
df

/home/anibal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,Triggers,Sense,Toehold Switch,dfull_sensor,dactive_sensor,dbinding_site,Dg_RBS_linker,score
0,GUACUGCCAACUGGAUCCUUCGCGGGACGUCCUUUG,Direct,CAAAGGACGUCCCGCGAAGGAUCCAGUUGGCAGUACGGACUUUAGA...,0.2359,0.3996,0.4646,-11.497,13.508462
1,UACUGCCAACUGGAUCCUUCGCGGGACGUCCUUUGU,Direct,ACAAAGGACGUCCCGCGAAGGAUCCAGUUGGCAGUAGGACUUUAGA...,0.2408,0.4228,0.4212,-12.380,12.556808
3,CUGCCAACUGGAUCCUUCGCGGGACGUCCUUUGUUU,Direct,AAACAAAGGACGUCCCGCGAAGGAUCCAGUUGGCAGGGACUUUAGA...,0.2550,0.4188,0.2199,-16.156,13.468384
4,UGCCAACUGGAUCCUUCGCGGGACGUCCUUUGUUUA,Direct,UAAACAAAGGACGUCCCGCGAAGGAUCCAGUUGGCAGGACUUUAGA...,0.2368,0.3887,0.2116,-9.100,16.301945
5,GCCAACUGGAUCCUUCGCGGGACGUCCUUUGUUUAC,Direct,GUAAACAAAGGACGUCCCGCGAAGGAUCCAGUUGGCGGACUUUAGA...,0.2356,0.3667,0.2153,-8.840,17.445741
6,CCAACUGGAUCCUUCGCGGGACGUCCUUUGUUUACG,Direct,CGUAAACAAAGGACGUCCCGCGAAGGAUCCAGUUGGGGACUUUAGA...,0.2079,0.4008,0.2241,-11.523,17.703088
7,CAACUGGAUCCUUCGCGGGACGUCCUUUGUUUACGU,Direct,ACGUAAACAAAGGACGUCCCGCGAAGGAUCCAGUUGGGACUUUAGA...,0.1995,0.3697,0.2864,-10.524,19.401181
8,AACUGGAUCCUUCGCGGGACGUCCUUUGUUUACGUC,Direct,GACGUAAACAAAGGACGUCCCGCGAAGGAUCCAGUUGGACUUUAGA...,0.2293,0.3707,0.3983,-11.023,16.195706
9,ACUGGAUCCUUCGCGGGACGUCCUUUGUUUACGUCC,Direct,GGACGUAAACAAAGGACGUCCCGCGAAGGAUCCAGUGGACUUUAGA...,0.2428,0.3770,0.4795,-10.446,13.920949
10,CUGGAUCCUUCGCGGGACGUCCUUUGUUUACGUCCC,Direct,GGGACGUAAACAAAGGACGUCCCGCGAAGGAUCCAGGGACUUUAGA...,0.2601,0.4222,0.5601,-13.239,9.187095


In [20]:
df.sort_values("score", ascending = False)
df

,Triggers,Sense,Toehold Switch,dfull_sensor,dactive_sensor,dbinding_site,Dg_RBS_linker,score
85,CUUGGGACUCUCUCGUCCCCUUCUCCGUCUGCCGUU,Direct,AACGGCAGACGGAGAAGGGGACGAGAGAGUCCCAAGGGACUUUAGA...,0.1631,0.3812,0.17250,-10.214,22.176495
84,GCUUGGGACUCUCUCGUCCCCUUCUCCGUCUGCCGU,Direct,ACGGCAGACGGAGAAGGGGACGAGAGAGUCCCAAGCGGACUUUAGA...,0.1610,0.3854,0.17250,-10.519,22.120845
86,UUGGGACUCUCUCGUCCCCUUCUCCGUCUGCCGUUC,Direct,GAACGGCAGACGGAGAAGGGGACGAGAGAGUCCCAAGGACUUUAGA...,0.1623,0.3882,0.17320,-10.935,21.886414
87,UGGGACUCUCUCGUCCCCUUCUCCGUCUGCCGUUCC,Direct,GGAACGGCAGACGGAGAAGGGGACGAGAGAGUCCCAGGACUUUAGA...,0.1698,0.3944,0.16480,-10.152,21.088605
152,ACACGGACCGGCAGAUGAGAAGGCACAGACGGGGAG,Reversed Complement,CUCCCCGUCUGUGCCUUCUCAUCUGCCGGUCCGUGUGGACUUUAGA...,0.2339,0.3200,0.06284,-5.994,20.600926
81,GCCGCUUGGGACUCUCUCGUCCCCUUCUCCGUCUGC,Direct,GCAGACGGAGAAGGGGACGAGAGAGUCCCAAGCGGCGGACUUUAGA...,0.1586,0.4262,0.11640,-16.207,20.564426
152,GCGGUCUCCCCGUCUGUGCCUUCUCAUCUGCCGGUC,Direct,GACCGGCAGAUGAGAAGGCACAGACGGGGAGACCGCGGACUUUAGA...,0.1772,0.3887,0.21480,-14.741,20.555869
88,GGGACUCUCUCGUCCCCUUCUCCGUCUGCCGUUCCA,Direct,UGGAACGGCAGACGGAGAAGGGGACGAGAGAGUCCCGGACUUUAGA...,0.1689,0.4109,0.14660,-11.814,20.435207
80,GGCCGCUUGGGACUCUCUCGUCCCCUUCUCCGUCUG,Direct,CAGACGGAGAAGGGGACGAGAGAGUCCCAAGCGGCCGGACUUUAGA...,0.1839,0.3923,0.12340,-13.940,20.383186
146,CUUUACGCGGUCUCCCCGUCUGUGCCUUCUCAUCUG,Direct,CAGAUGAGAAGGCACAGACGGGGAGACCGCGUAAAGGGACUUUAGA...,0.1870,0.3869,0.14170,-9.252,20.339784


## Save the output to a CSV

In [12]:
df.to_csv("outputs/Ranked_designs.csv")